In [1]:
import json
import random
from collections import defaultdict
from itertools import chain
from typing import Iterable

import keras
import numpy
import pandas
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, StratifiedKFold

Using TensorFlow backend.


In [2]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scoring = 'accuracy'

In [3]:
# Copy-pasted from the game.
# https://heroes.cdnvideo.ru/vk/v0326/locale/ru.json.gz

lib = {
  "0": "Карен",
  "1": "Аврора",
  "10": "Безликий",
  "11": "Чабба",
  "12": "Арахна",
  "13": "Орион",
  "14": "Фокс",
  "15": "Джинджер",
  "16": "Данте",
  "17": "Моджо",
  "18": "Судья",
  "19": "Темная Звезда",
  "2": "Галахад",
  "20": "Артемис",
  "21": "Маркус",
  "22": "Пеппи",
  "23": "Лиэн",
  "24": "Тесак",
  "25": "Исмаил",
  "26": "Лилит",
  "27": "Лютер",
  "28": "Цин Мао",
  "29": "Дориан",
  "3": "Кира",
  "30": "Корнелиус",
  "31": "Джет",
  "32": "Гелиос",
  "33": "Ларс",
  "34": "Криста",
  "35": "Йорген",
  "36": "Майя",
  "37": "Джу",
  "38": "Эльмир",
  "39": "Зири",
  "4": "Астарот",
  "40": "Небула",
  "41": "К'арх",
  "5": "Кай",
  "6": "Фобос",
  "7": "Тея",
  "8": "Сорвиголова",
  "9": "Хайди",
}

In [4]:
# Parse battle logs.

def parse_heroes(heroes: Iterable[dict], sign: int, result: dict):
    for hero in heroes:
        for hero_key in ('Level', 'Color', 'Star'):
            result[f'''{lib[str(hero['id'])]} {hero_key}'''] += sign * hero[hero_key.lower()]


def parse_battle(line: str) -> dict:
    battle = json.loads(line)
    result = defaultdict(int)

    parse_heroes(battle.get('attackers') or battle['player'], +1, result)
    parse_heroes(battle.get('defenders') or battle['enemies'], -1, result)
        
    return {'Win': battle['win'], **result}

In [5]:
# Load the logs into a data frame.

def invert_column(series: pandas.Series):
    """
    Inverts the column to make an "opposite" battle.
    """
    return series == False if series.name == 'Win' else -series

lines = list(chain.from_iterable([
    open('battles.jsonl'),
    open('battles-lilia.jsonl'),
]))

numpy.random.seed(42)
battles = pandas.DataFrame([parse_battle(line) for line in lines]).fillna(value=0)
battles.drop_duplicates(inplace=True)  # because the files may contain duplicate battles
battles = pandas.concat((battles, battles.apply(invert_column)))  # add inverted battles
battles = battles.sample(frac=1)  # shuffle
battles.head()

,Win,Аврора Color,Аврора Level,Аврора Star,Арахна Color,Арахна Level,Арахна Star,Артемис Color,Артемис Level,Артемис Star,...,Хайди Star,Цин Мао Color,Цин Мао Level,Цин Мао Star,Чабба Color,Чабба Level,Чабба Star,Эльмир Color,Эльмир Level,Эльмир Star
152,False,-0.0,-0.0,-0.0,-5.0,-43.0,-3.0,-0.0,-0.0,-0.0,...,1.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-6.0,-43.0,-3.0
33,False,0.0,0.0,0.0,-2.0,-6.0,-1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,False,-5.0,-46.0,-1.0,4.0,37.0,2.0,-5.0,-50.0,-2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
139,True,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
57,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-4.0,-45.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
battles.describe()

,Аврора Color,Аврора Level,Аврора Star,Арахна Color,Арахна Level,Арахна Star,Артемис Color,Артемис Level,Артемис Star,Астарот Color,...,Хайди Star,Цин Мао Color,Цин Мао Level,Цин Мао Star,Чабба Color,Чабба Level,Чабба Star,Эльмир Color,Эльмир Level,Эльмир Star
count,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.00000,...,374.000000,374.000000,374.000000,374.000000,374.00000,374.000000,374.0000,374.000000,374.000000,374.000000
mean,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0000,0.000000,0.000000,0.000000
std,2.502546,19.797365,0.896822,3.558775,28.321498,1.732825,3.517097,29.199508,1.289263,5.13778,...,0.358729,1.320086,11.069365,0.401071,2.00134,16.121522,0.8508,0.923335,7.502993,0.407701
min,-9.000000,-70.000000,-3.000000,-9.000000,-71.000000,-6.000000,-8.000000,-69.000000,-3.000000,-9.00000,...,-2.000000,-8.000000,-71.000000,-3.000000,-8.00000,-70.000000,-3.0000,-7.000000,-60.000000,-3.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-6.00000,...,0.000000,0.000000,0.000000,0.000000,-0.00000,-0.000000,-0.0000,0.000000,0.000000,0.000000
50%,-0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,-0.000000,-0.00000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,6.00000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0000,0.000000,0.000000,0.000000
max,9.000000,70.000000,3.000000,9.000000,71.000000,6.000000,8.000000,69.000000,3.000000,9.00000,...,2.000000,8.000000,71.000000,3.000000,8.00000,70.000000,3.0000,7.000000,60.000000,3.000000


In [7]:
x = battles.drop(['Win'], axis=1)
y = battles['Win']

In [8]:
classifier = LogisticRegression(max_iter=1000, fit_intercept=False)
param_grid = {'C': numpy.logspace(-6, 2, num=1000)}

numpy.random.seed(42)
grid_search = GridSearchCV(classifier, param_grid, cv=cv, scoring=scoring, n_jobs=4).fit(x, y)

print(f'Best score: {grid_search.best_score_}')
print(f'Best params: {grid_search.best_params_}')
print(f'Classes: {grid_search.best_estimator_.classes_}')

scores = cross_val_score(grid_search.best_estimator_, x, y, scoring=scoring, cv=cv)
print(f'CV score: {scores.mean()} (std: {scores.std()})')

Best score: 0.983957219251337
Best params: {'C': 78.68571506936844}
Classes: [False  True]
CV score: 0.9815789473684211 (std: 0.02368421052631581)


In [9]:
pandas.DataFrame({'Feature': x.columns, 'Importance': grid_search.best_estimator_.coef_[0]}) \
    .set_index('Feature') \
    .sort_values('Importance', ascending=False) \
    .head(n=20)

,Importance
Feature,
Криста Color,14.197941
Астарот Star,11.471885
Зири Color,10.325526
Аврора Star,9.845360
Судья Star,9.726217
Фобос Color,7.313225
Криста Star,6.064186
Исмаил Star,5.874418
Орион Star,5.759005


In [10]:
result = pandas.concat((
    pandas.Series(grid_search.best_estimator_.predict(x), index=battles.index, name='Predicted'),
    pandas.Series(grid_search.best_estimator_.predict_proba(x)[:, 1], index=battles.index, name='Probability'),
    battles,
), axis=1)
result['Probability'] = result['Probability'].apply('{:.2f}'.format)
result.head()

,Predicted,Probability,Win,Аврора Color,Аврора Level,Аврора Star,Арахна Color,Арахна Level,Арахна Star,Артемис Color,...,Хайди Star,Цин Мао Color,Цин Мао Level,Цин Мао Star,Чабба Color,Чабба Level,Чабба Star,Эльмир Color,Эльмир Level,Эльмир Star
152,False,0.00,False,-0.0,-0.0,-0.0,-5.0,-43.0,-3.0,-0.0,...,1.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-6.0,-43.0,-3.0
33,False,0.00,False,0.0,0.0,0.0,-2.0,-6.0,-1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,False,0.00,False,-5.0,-46.0,-1.0,4.0,37.0,2.0,-5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
139,True,1.00,True,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
57,True,1.00,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-4.0,-45.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Experimental NN model.
# I want it to catch an interaction between different heroes (e.g. Йорген is good in conjunction with Исмаил).

def create_model() -> keras.Model:
    model = Sequential()
    model.add(Dense(1000, input_dim=x.columns.size, activation='sigmoid'))
    model.add(Dropout(rate=0.1))
    model.add(Dense(1, input_dim=x.columns.size, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

keras_classifier = KerasClassifier(build_fn=create_model, epochs=100, verbose=0)

numpy.random.seed(42)
scores = cross_val_score(keras_classifier, x, y, cv=cv)

print(f'Score: {scores.mean()} (std: {scores.std()})')

Score: 0.97105263095153 (std: 0.04625893743310146)
